In [1]:
import numpy as np
import pandas as pd 
import os
from utils import *
from model import *

In [2]:
import numpy as np
import random
import math
import os
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pandas as pd
import glob
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.autograd import Variable
import scipy.io
import functools

%matplotlib inline

from math import sqrt
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error

from scipy.signal import find_peaks

from math import sqrt

# Read data

In [3]:
concat_df = pd.read_csv("./data/concat_df.csv")
rest_df = pd.read_csv("./data/rest_df.csv")

In [4]:
_concat_df = {}
_rest_df = {}
batteries = [5,6,7,18]
for bat in batteries:
    _concat_df[bat] = concat_df[concat_df['battery_nb'] == bat].copy().reset_index()
    _rest_df[bat] = rest_df[rest_df['battery_nb'] == bat].copy().reset_index()
del concat_df, rest_df

## Global feature

In [5]:
global_df = {}
for bat in batteries:
    global_df[bat] = _concat_df[bat].groupby(['charge_nb'],as_index=False).mean()[['charge_nb','interpolated_capacity']].sort_values(['charge_nb'])
    global_df[bat]['mean_previous_5'] = global_df[bat]['interpolated_capacity'].rolling(5).mean().shift(1).fillna(10)
    global_df[bat]['isPeak'] = np.where(global_df[bat]['interpolated_capacity'] > global_df[bat]['mean_previous_5'],1,0)
    global_df[bat]['x2'] = _rest_df[bat]['cc_capacity'].copy()
    global_df[bat]['x3'] = _rest_df[bat]['cv_capacity'].copy()
    temp_df = _concat_df[bat].groupby(['charge_nb']).last().reset_index()
    global_df[bat]['x4'] = temp_df['current_measured'].copy()
    global_df[bat]['x5'] = temp_df['voltage_measured'].copy()
    del temp_df
    global_df[bat].drop(['mean_previous_5'],axis=1,inplace=True)

<ipython-input-5-2b08fb307b9e>:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  global_df[bat] = _concat_df[bat].groupby(['charge_nb'],as_index=False).mean()[['charge_nb','interpolated_capacity']].sort_values(['charge_nb'])
<ipython-input-5-2b08fb307b9e>:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  global_df[bat] = _concat_df[bat].groupby(['charge_nb'],as_index=False).mean()[['charge_nb','interpolated_capacity']].sort_values(['charge_nb'])
<ipython-input-5-2b08fb307b9e>:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will defa

In [6]:
global_df[5].head()

,charge_nb,interpolated_capacity,isPeak,x2,x3,x4,x5
0,1.0,1.856487,0,0.995578,0.0,-0.002892,4.191078
1,2.0,1.846327,0,0.995986,0.0,-0.001833,4.189062
2,3.0,1.835349,0,0.995144,0.0,0.002277,4.187398
3,4.0,1.835263,0,0.995155,0.0,-0.000862,4.188055
4,5.0,1.834646,0,0.993751,0.0,0.000398,4.188438


In [7]:
# normalize data
column_to_norm = ['voltage_measured','current_measured','temperature_measured']
max_values = {}
min_values = {}

full_df = pd.concat([_concat_df[bat] for bat in _concat_df],ignore_index=True)
for col in column_to_norm:
  max_values[col] =  full_df[col].max()
  min_values[col] =  full_df[col].min()
  for bat in _concat_df:
      _concat_df[bat][col]=(_concat_df[bat][col]-min_values[col])/(max_values[col]-min_values[col]) 

column_to_norm = ['x2','x3','x4','x5', 'interpolated_capacity']
full_df = pd.concat([global_df[bat] for bat in global_df],ignore_index=True)
for col in column_to_norm:
  max_values[col] =  full_df[col].max()
  min_values[col] =  full_df[col].min()
  for bat in _concat_df:
      global_df[bat][col]=(global_df[bat][col]-min_values[col])/(max_values[col]-min_values[col]) 

In [8]:
window_size=10
prediction_interval=1
multichannel_features = ['voltage_measured','current_measured','temperature_measured']
# global_seq_features = ['x_5', 'rest_period', 'interpolated_capacity']
global_seq_features = ['x2','x3','x4','x5', 'isPeak', 'interpolated_capacity']
extract_points_len = 11

In [9]:
import numpy as np
import pickle
import torch
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

In [11]:
class Obj(object):
    def __init__(self):
        pass
    
args = Obj()
    
args.data_path = "./data/data.pkl"
args.batch_size = 100
args.shuffle = True
args.pin_memory = True
args.embed_dim = 50
args.hidden_dim = 16
args.dropout = 0.06
args.learning_rate = 0.02
args.device = 'cpu'
args.num_epochs = 2000
args.summary_batch_interval = 1


sequence_feature_count = 3
global_feature_count = 6
num_points = 11
num_cycles = 10
prediction_interval = 1
maximum = max_values['interpolated_capacity']
minimum = min_values['interpolated_capacity']

In [15]:
def build_and_train(model, out_dir, data, EPOCH, seed,dropout, clipping_value=None, park=False):
    setup_seed(seed)
    
    loss_list, y_ = [0], [] 
    rmse_list, mape_list = [], []
    best_rmse = None
#     model = CNN_Transformer_ARNS(feature_size, num_extracted_point, window_size, global_feature_size, frt_feature_size, output_node,
#                  dccnn_kernel_size, dccnn_stride, dccnn_padding, dccnn_dilation, 
#                  dropout, 
#                  pcnn_kernel_size, pcnn_stride, pcnn_padding, pcnn_dilation, 
#                  n_layers)
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=weight_decay)
    criterion = nn.MSELoss()
    
#     if park:
#         X_train, Y_train, X_val, Y_val, X_test,Y_test = park_cross_validation(batteries, concat_df)
#     else:
#         X_train, Y_train, X_val, Y_val, X_test,Y_test = cross_validation(batteries, concat_df)

    X_train, Y_train, X_val, Y_val, X_test,Y_test = data
    
    print('sample size: {}'.format(len(X_train[0])))
    print('target size: {}'.format(len(Y_train)))
    R_avg = []
    for epoch in range(EPOCH) :
        model.train()
        X_local, X_global = X_train
        X_local = np.array(X_local).astype(np.float32)
        X_global = np.array(X_global).astype(np.float32)
        Y = np.array(Y_train).astype(np.float32)
        X_local, X_global = torch.from_numpy(X_local).to(device), torch.from_numpy(X_global).to(device) 
        Y = torch.from_numpy(Y).to(device)
        output, attention_weights = model(X_local, X_global)
#         rl_loss = 0
        downstream_loss = criterion(output, Y)
        loss = downstream_loss
        optimizer.zero_grad()
        loss.backward()
        if clipping_value:
            nn.utils.clip_grad_norm(model.parameters(), clipping_value)
        optimizer.step()

        if epoch%100==0 or epoch==(EPOCH-1):
            point_list = []
            x_local, x_global = X_val
            x_local = np.array(x_local).astype(np.float32)
            x_global = np.array(x_global).astype(np.float32)
            x_local, x_global,  = torch.from_numpy(x_local).to(device), torch.from_numpy(x_global).to(device)
            model.eval()
            with torch.no_grad():
                pred, _ = model(x_local, x_global)
#                 print(pred)
            point_list = list(pred[:,0].cpu().detach().numpy())
            y_.append(point_list)
            loss_list.append(loss)
            mape, rmse = evaluation(y_test=denormalize(Y_val,maximum,minimum).flatten(), y_predict=denormalize(np.array(y_[-1]),maximum,minimum))
            mape_list.append(mape)
            rmse_list.append(rmse)
            if best_rmse == None:
                best_rmse = {}
                best_rmse['name'] = 'checkpoint-' + str(epoch) + '.pt'
                best_rmse['value'] = rmse
                torch.save(model, os.path.join(out_dir,best_rmse['name']))
            else:
                if rmse < best_rmse['value']:
                    os.remove(os.path.join(out_dir,best_rmse['name']))
                    best_rmse['name'] = 'checkpoint-' + str(epoch) + '.pt'
                    best_rmse['value'] = rmse
                    torch.save(model, os.path.join(out_dir,best_rmse['name']))
            re = relative_error(y_test=Y_val.flatten(), y_predict=y_[-1], threshold=0.7)
            print('epoch:{:<2d} | loss:{:<6.7f} | MAPE:{:<6.4f} | RMSE:{:<6.4f} | RE:{:<6.4f}'.format(epoch, loss, mape, rmse, re))
            
        if (len(loss_list) > 1) and (abs(loss_list[-2] - loss_list[-1]) < 1e-7):
          break
    mape, rmse = evaluation(y_test=denormalize(Y_val,maximum,minimum).flatten(), y_predict=denormalize(np.array(y_[-1]),maximum,minimum))
    re = relative_error(y_test=Y_val.flatten(), y_predict=y_[-1], threshold=0.7)
    return model, y_[-1], mape_list, rmse_list, re

## Cross 1

In [16]:
ver = 2
device = "cpu"
LR = 0.02
weight_decay = 0

In [18]:
cross = [0.2, 0.4, 0.7, 1, 0.2, 0.4, 0.7, 1, 0.2, 0.4, 0.7, 1, 0.2, 0.4, 0.7, 1,]

size = {}
size[5] = len(global_df[5])
size[6] = len(global_df[6])
size[7] = len(global_df[7])
size[18] = len(global_df[18])

def split_set(index):
    X_test, Y_test = [[],[]], []
    for bat in index: 
        for (fr,to) in index[bat]:
            X, Y = extract_batter_features(_concat_df[bat].loc[int(fr*size[bat])*num_points:int(to*size[bat])*num_points+num_points-1],global_df[bat].loc[int(fr*size[bat]):int(to*size[bat])],bat,num_cycles,prediction_interval,multichannel_features,extract_points_len,global_seq_features)
            X_test[0].append(X[0])
            X_test[1].append(X[1])
            Y_test.append(Y)
    X_test[0] = np.concatenate(X_test[0],axis=0)
    X_test[1] = np.concatenate(X_test[1],axis=0)
    Y_test = np.concatenate(Y_test,axis=0)
    return X_test, Y_test

index = {}
index[5] = [(0,0.2)]
index[6] = [(0.2,0.4)]
index[7] = [(0.4,0.7)]
index[18] = [(0.7,1.)]
X_test, Y_test = split_set(index)

index = {}
index[5] = [(0.2,0.7)]
index[6] = [(0,0.2),(0.4,0.7)]
index[7] = [(0,0.4)]
index[18] = [(0,0.4)]
X_train, Y_train = split_set(index)

index = {}
index[5] = [(0.7,1)]
index[6] = [(0.7,1)]
index[7] = [(0.7,1)]
index[18] = [(0.4,0.7)]
X_val, Y_val = split_set(index)

/home/dh2409/Documents/LIB/fixed-point-model/utils.py:17: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  capacity = df_copy.groupby(['charge_nb']).mean()['interpolated_capacity'].to_numpy().reshape(-1, 1)
/home/dh2409/Documents/LIB/fixed-point-model/utils.py:17: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  capacity = df_copy.groupby(['charge_nb']).mean()['interpolated_capacity'].to_numpy().reshape(-1, 1)
/home/dh2409/Documents/LIB/fixed-point-model/utils.py:17: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Ei

In [20]:
path = 'cross1_v'+str(ver)
if not os.path.isdir(path):
    os.mkdir(path)
model = Regressor('bert-trainposcode', 'LSTM_bi', True, 1, args.embed_dim, args.hidden_dim, args.dropout)
print(f"Trainable params: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")
_,_,_,_,_ = build_and_train(model,path,(X_train,Y_train,X_val,Y_val,X_test,Y_test),args.num_epochs,0,args.dropout)

Trainable params: 19999
sample size: 239
target size: 239
epoch:0  | loss:1.9384538 | MAPE:4.1867 | RMSE:0.1157 | RE:1.0000
epoch:100 | loss:0.0071739 | MAPE:1.6914 | RMSE:0.0433 | RE:1.0000
epoch:200 | loss:0.0045110 | MAPE:1.6901 | RMSE:0.0431 | RE:1.0000
epoch:300 | loss:0.0030270 | MAPE:1.2902 | RMSE:0.0338 | RE:1.0000
epoch:400 | loss:0.0024175 | MAPE:0.9534 | RMSE:0.0264 | RE:1.0000
epoch:500 | loss:0.0014291 | MAPE:0.7917 | RMSE:0.0228 | RE:1.0000
epoch:600 | loss:0.0011632 | MAPE:0.6334 | RMSE:0.0193 | RE:1.0000
epoch:700 | loss:0.0008199 | MAPE:0.5817 | RMSE:0.0179 | RE:1.0000
epoch:800 | loss:0.0006342 | MAPE:0.5255 | RMSE:0.0169 | RE:1.0000
epoch:900 | loss:0.0005123 | MAPE:0.6225 | RMSE:0.0191 | RE:1.0000
epoch:1000 | loss:0.0005450 | MAPE:0.6987 | RMSE:0.0210 | RE:1.0000
epoch:1100 | loss:0.0004297 | MAPE:0.9535 | RMSE:0.0274 | RE:1.0000
epoch:1200 | loss:0.0003566 | MAPE:1.1092 | RMSE:0.0315 | RE:1.0000
epoch:1300 | loss:0.0003179 | MAPE:1.3289 | RMSE:0.0389 | RE:1.0000
e

In [22]:
path_model = os.listdir(path)[0]
model = torch.load(os.path.join(path,path_model))
mape1,mae1,rmse1 = test(model, X_test, Y_test, starting_point=0,maximum=maximum,minimum=minimum)

MAPE 0.7158543890577939 
MAE 0.018229498340099265 
RSME 0.024549771036934672
-----------------------------------------


## Cross 2

In [23]:
ver = 2
device = "cpu"
LR = 0.02
weight_decay = 0

In [24]:
cross = [0.2, 0.4, 0.7, 1, 0.2, 0.4, 0.7, 1, 0.2, 0.4, 0.7, 1, 0.2, 0.4, 0.7, 1,]

size = {}
size[5] = len(global_df[5])
size[6] = len(global_df[6])
size[7] = len(global_df[7])
size[18] = len(global_df[18])

index = {}
index[5] = [(0,0.2)]
index[6] = [(0.2,0.4)]
index[7] = [(0.4,0.7)]
index[18] = [(0.7,1.)]
X_test, Y_test = split_set(index)

index = {}
index[5] = [(0.2,0.4),(0.7,1)]
index[6] = [(0,0.2),(0.7,1)]
index[7] = [(0,0.2),(0.7,1)]
index[18] = [(0,0.2),(0.4,0.7)]
X_train, Y_train = split_set(index)

index = {}
index[5] = [(0.4,0.7)]
index[6] = [(0.4,0.7)]
index[7] = [(0.2,0.4)]
index[18] = [(0.2,0.4)]
X_val, Y_val = split_set(index)

/home/dh2409/Documents/LIB/fixed-point-model/utils.py:17: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  capacity = df_copy.groupby(['charge_nb']).mean()['interpolated_capacity'].to_numpy().reshape(-1, 1)
/home/dh2409/Documents/LIB/fixed-point-model/utils.py:17: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  capacity = df_copy.groupby(['charge_nb']).mean()['interpolated_capacity'].to_numpy().reshape(-1, 1)
/home/dh2409/Documents/LIB/fixed-point-model/utils.py:17: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Ei

In [25]:
path = 'cross2_v'+str(ver)
if not os.path.isdir(path):
    os.mkdir(path)
model = Regressor('bert-trainposcode', 'LSTM_bi', True, 1, args.embed_dim, args.hidden_dim, args.dropout)
print(f"Trainable params: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")
_,_,_,_,_ = build_and_train(model,path,(X_train,Y_train,X_val,Y_val,X_test,Y_test),args.num_epochs,0,args.dropout)

Trainable params: 19999
sample size: 236
target size: 236
epoch:0  | loss:1.2190431 | MAPE:6.6412 | RMSE:0.1838 | RE:1.0000
epoch:100 | loss:0.0041358 | MAPE:0.7268 | RMSE:0.0274 | RE:1.0000
epoch:200 | loss:0.0020799 | MAPE:0.7893 | RMSE:0.0292 | RE:1.0000
epoch:300 | loss:0.0014406 | MAPE:0.7812 | RMSE:0.0295 | RE:1.0000
epoch:400 | loss:0.0010139 | MAPE:0.7185 | RMSE:0.0301 | RE:1.0000
epoch:500 | loss:0.0007971 | MAPE:0.6878 | RMSE:0.0281 | RE:1.0000
epoch:600 | loss:0.0006545 | MAPE:0.6603 | RMSE:0.0272 | RE:1.0000
epoch:700 | loss:0.0005852 | MAPE:0.6899 | RMSE:0.0284 | RE:1.0000
epoch:800 | loss:0.0005334 | MAPE:0.6466 | RMSE:0.0259 | RE:1.0000
epoch:900 | loss:0.0005227 | MAPE:0.6036 | RMSE:0.0254 | RE:1.0000
epoch:1000 | loss:0.0004565 | MAPE:0.6171 | RMSE:0.0244 | RE:1.0000
epoch:1100 | loss:0.0004298 | MAPE:0.5721 | RMSE:0.0254 | RE:1.0000
epoch:1200 | loss:0.0003987 | MAPE:0.6089 | RMSE:0.0247 | RE:1.0000
epoch:1300 | loss:0.0003595 | MAPE:0.5573 | RMSE:0.0252 | RE:1.0000
e

In [26]:
path_model = os.listdir(path)[0]
model = torch.load(os.path.join(path,path_model))
mape2,mae2,rmse2 = test(model, X_test, Y_test, starting_point=0,maximum=maximum,minimum=minimum)

MAPE 0.7262825874913497 
MAE 0.01900688562126774 
RSME 0.026668593330766627
-----------------------------------------


# Cross 3

In [27]:
ver = 2
device = "cpu"
LR = 0.02
weight_decay = 0

In [28]:
cross = [0.2, 0.4, 0.7, 1, 0.2, 0.4, 0.7, 1, 0.2, 0.4, 0.7, 1, 0.2, 0.4, 0.7, 1,]

size = {}
size[5] = len(global_df[5])
size[6] = len(global_df[6])
size[7] = len(global_df[7])
size[18] = len(global_df[18])

index = {}
index[5] = [(0,0.2)]
index[6] = [(0.2,0.4)]
index[7] = [(0.4,0.7)]
index[18] = [(0.7,1.)]
X_test, Y_test = split_set(index)

index = {}
index[5] = [(0.4,1)]
index[6] = [(0.4,1)]
index[7] = [(0.2,0.4),(0.7,1)]
index[18] = [(0.2,0.7)]
X_train, Y_train = split_set(index)

index = {}
index[5] = [(0.2,0.4)]
index[6] = [(0,0.2)]
index[7] = [(0,0.2)]
index[18] = [(0,0.2)]
X_val, Y_val = split_set(index)

/home/dh2409/Documents/LIB/fixed-point-model/utils.py:17: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  capacity = df_copy.groupby(['charge_nb']).mean()['interpolated_capacity'].to_numpy().reshape(-1, 1)
/home/dh2409/Documents/LIB/fixed-point-model/utils.py:17: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  capacity = df_copy.groupby(['charge_nb']).mean()['interpolated_capacity'].to_numpy().reshape(-1, 1)
/home/dh2409/Documents/LIB/fixed-point-model/utils.py:17: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Ei

In [ ]:
path = 'cross3_v'+str(ver)
if not os.path.isdir(path):
    os.mkdir(path)
model = Regressor('bert-trainposcode', 'LSTM_bi', True, 1, args.embed_dim, args.hidden_dim, args.dropout)
print(f"Trainable params: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")
_,_,_,_,_ = build_and_train(model,path,(X_train,Y_train,X_val,Y_val,X_test,Y_test),args.num_epochs,0,args.dropout)

Trainable params: 19999
sample size: 303
target size: 303
epoch:0  | loss:1.2937106 | MAPE:19.4799 | RMSE:0.5465 | RE:1.0000
epoch:100 | loss:0.0057470 | MAPE:1.0234 | RMSE:0.0371 | RE:1.0000
epoch:200 | loss:0.0032956 | MAPE:0.9485 | RMSE:0.0349 | RE:1.0000
epoch:300 | loss:0.0016259 | MAPE:0.9511 | RMSE:0.0345 | RE:1.0000
epoch:400 | loss:0.0012323 | MAPE:0.9031 | RMSE:0.0330 | RE:1.0000
epoch:500 | loss:0.0008173 | MAPE:0.8190 | RMSE:0.0312 | RE:1.0000
epoch:600 | loss:0.0005740 | MAPE:0.8187 | RMSE:0.0312 | RE:1.0000
epoch:700 | loss:0.0004557 | MAPE:0.8126 | RMSE:0.0313 | RE:1.0000
epoch:800 | loss:0.0004144 | MAPE:0.7851 | RMSE:0.0305 | RE:1.0000
epoch:900 | loss:0.0003602 | MAPE:0.7680 | RMSE:0.0301 | RE:1.0000
epoch:1000 | loss:0.0003806 | MAPE:0.7599 | RMSE:0.0300 | RE:1.0000
epoch:1100 | loss:0.0003614 | MAPE:0.7582 | RMSE:0.0297 | RE:1.0000
epoch:1200 | loss:0.0003555 | MAPE:0.7375 | RMSE:0.0297 | RE:1.0000
epoch:1300 | loss:0.0003538 | MAPE:0.7354 | RMSE:0.0294 | RE:1.0000


In [ ]:
path_model = os.listdir(path)[0]
model = torch.load(os.path.join(path,path_model))
mape3,mae3,rmse3 = test(model, X_test, Y_test, starting_point=0,maximum=maximum,minimum=minimum)

In [ ]:
print("Mean of rmse:",(rmse1+rmse2+rmse3)/3)
print("Mean of mape:",(mape1+mape2+mape3)/3)